Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
pickle_file = './data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [32]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [33]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [34]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

W1006 17:11:42.076472 14412 deprecation.py:323] From <ipython-input-34-3b42a02b2811>:45: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [35]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.164938
Minibatch accuracy: 12.5%
Validation accuracy: 10.2%
Minibatch loss at step 50: 1.541246
Minibatch accuracy: 43.8%
Validation accuracy: 57.9%
Minibatch loss at step 100: 1.022344
Minibatch accuracy: 68.8%
Validation accuracy: 69.7%
Minibatch loss at step 150: 0.528188
Minibatch accuracy: 81.2%
Validation accuracy: 74.4%
Minibatch loss at step 200: 0.987575
Minibatch accuracy: 75.0%
Validation accuracy: 77.0%
Minibatch loss at step 250: 1.138136
Minibatch accuracy: 68.8%
Validation accuracy: 76.5%
Minibatch loss at step 300: 0.356399
Minibatch accuracy: 81.2%
Validation accuracy: 79.5%
Minibatch loss at step 350: 0.469874
Minibatch accuracy: 93.8%
Validation accuracy: 76.6%
Minibatch loss at step 400: 0.289099
Minibatch accuracy: 93.8%
Validation accuracy: 80.0%
Minibatch loss at step 450: 0.750240
Minibatch accuracy: 81.2%
Validation accuracy: 77.8%
Minibatch loss at step 500: 0.542907
Minibatch accuracy: 87.5%
Validation accuracy: 80.9%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [45]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    # substitute stride of 2 to 1
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    # remove the relu layer
    hidden = tf.nn.relu(conv + layer1_biases)
    # add a max pooling layer
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1,2,2,1], padding='SAME')
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    # add the second pooling layer
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1,2,2,1], padding='SAME')
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [46]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.525509
Minibatch accuracy: 6.2%
Validation accuracy: 7.1%
Minibatch loss at step 50: 1.815562
Minibatch accuracy: 31.2%
Validation accuracy: 46.3%
Minibatch loss at step 100: 1.208352
Minibatch accuracy: 56.2%
Validation accuracy: 59.6%
Minibatch loss at step 150: 0.393738
Minibatch accuracy: 87.5%
Validation accuracy: 73.8%
Minibatch loss at step 200: 0.940015
Minibatch accuracy: 75.0%
Validation accuracy: 76.9%
Minibatch loss at step 250: 1.243312
Minibatch accuracy: 68.8%
Validation accuracy: 76.6%
Minibatch loss at step 300: 0.415548
Minibatch accuracy: 87.5%
Validation accuracy: 79.9%
Minibatch loss at step 350: 0.532799
Minibatch accuracy: 93.8%
Validation accuracy: 78.1%
Minibatch loss at step 400: 0.267659
Minibatch accuracy: 100.0%
Validation accuracy: 80.4%
Minibatch loss at step 450: 0.956002
Minibatch accuracy: 75.0%
Validation accuracy: 79.9%
Minibatch loss at step 500: 0.681936
Minibatch accuracy: 87.5%
Validation accuracy: 81.3%
Mi

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [76]:
tf.nn.dropout?

In [99]:
batch_size = 128
patch_size = 5
depth = 16
num_hidden = 64
train_keep_rate = 1.0

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [4 * 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  added_conv_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  added_conv_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  
  
  # Model.
  def model(data, keep_rate):
    # substitute stride of 2 to 1
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    # add a max pooling layer
    pool = tf.nn.avg_pool(hidden, ksize=[1, 2, 2, 1], strides=[1,2,2,1], padding='SAME')
#     pool = tf.nn.dropout(pool, drop_rate)
    
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    # add the second pooling layer
    pool = tf.nn.avg_pool(hidden, ksize=[1, 2, 2, 1], strides=[1,2,2,1], padding='SAME')
#     pool = tf.nn.dropout(pool, drop_rate)
    
    # add the third convolution layer
    conv = tf.nn.conv2d(pool, added_conv_weights, [1, 1, 1, 1], padding='SAME')
    pool = tf.nn.avg_pool(conv, ksize=[1, 2, 2, 1], strides=[1,2,2,1], padding='SAME')
#     pool = tf.nn.dropout(pool, drop_rate)
    
    shape = pool.get_shape().as_list()
    print ("shape after two conv layer: {}".format(shape))
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    print ("shape after reshape: {}".format(shape))
    print ("layer_3_weights: {}".format(layer3_weights.shape))
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    hidden = tf.nn.dropout(hidden, keep_rate)
    
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset, train_keep_rate)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
#   optimizer = tf.train.AdamOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, keep_rate=1.0))
  test_prediction = tf.nn.softmax(model(tf_test_dataset, keep_rate=1.0))

shape after two conv layer: [128, 4, 4, 16]
shape after reshape: [128, 4, 4, 16]
layer_3_weights: (256, 64)
shape after two conv layer: [10000, 4, 4, 16]
shape after reshape: [10000, 4, 4, 16]
layer_3_weights: (256, 64)
shape after two conv layer: [10000, 4, 4, 16]
shape after reshape: [10000, 4, 4, 16]
layer_3_weights: (256, 64)


In [100]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.234791
Minibatch accuracy: 10.9%
Validation accuracy: 9.4%
Minibatch loss at step 50: 2.088753
Minibatch accuracy: 28.1%
Validation accuracy: 41.8%
Minibatch loss at step 100: 1.602081
Minibatch accuracy: 54.7%
Validation accuracy: 55.5%
Minibatch loss at step 150: 1.367667
Minibatch accuracy: 60.2%
Validation accuracy: 64.9%
Minibatch loss at step 200: 1.008186
Minibatch accuracy: 73.4%
Validation accuracy: 68.1%
Minibatch loss at step 250: 0.880972
Minibatch accuracy: 79.7%
Validation accuracy: 72.7%
Minibatch loss at step 300: 0.935558
Minibatch accuracy: 74.2%
Validation accuracy: 75.2%
Minibatch loss at step 350: 0.788497
Minibatch accuracy: 81.2%
Validation accuracy: 75.5%
Minibatch loss at step 400: 0.918280
Minibatch accuracy: 71.9%
Validation accuracy: 77.3%
Minibatch loss at step 450: 0.818325
Minibatch accuracy: 78.1%
Validation accuracy: 77.5%
Minibatch loss at step 500: 0.620478
Minibatch accuracy: 83.6%
Validation accuracy: 78.6%
Mi

In [98]:
## Todo: optimize the results